<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Main-Cycle" data-toc-modified-id="Main-Cycle-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Main Cycle</a></span><ul class="toc-item"><li><span><a href="#Generate-Mapping" data-toc-modified-id="Generate-Mapping-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Generate Mapping</a></span></li><li><span><a href="#Process-Documents" data-toc-modified-id="Process-Documents-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Process Documents</a></span><ul class="toc-item"><li><span><a href="#My-matheuristic-Papers" data-toc-modified-id="My-matheuristic-Papers-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>My matheuristic Papers</a></span></li><li><span><a href="#From-Arxiv.org" data-toc-modified-id="From-Arxiv.org-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>From Arxiv.org</a></span></li></ul></li></ul></li><li><span><a href="#Doc2vec-Creation" data-toc-modified-id="Doc2vec-Creation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Doc2vec Creation</a></span></li><li><span><a href="#Using-the-doc2vec-model" data-toc-modified-id="Using-the-doc2vec-model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Using the doc2vec model</a></span></li><li><span><a href="#Populate-Lists" data-toc-modified-id="Populate-Lists-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Populate Lists</a></span></li></ul></div>

In [9]:
import io
import os
#pdf2txt !!!!
import PyPDF2 as pyPdf

from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.converter import PDFPageAggregator
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LTTextBoxHorizontal

import re
import numpy as np
import pandas as pd
from collections import namedtuple
import matplotlib.pyplot as plt

import shutil
from os import path, listdir
from nltk import word_tokenize
from nltk.util import ngrams
import itertools
import pickle
import glob

# remove plurals
import inflect
p = inflect.engine()
#p.classical() # switch on classical plurals. E.g.: focus --> foci
p.defnoun( "class", "class" )
p.defnoun( "basis", "basis" )
p.defnoun("numerous", "numerous")
p.defnoun("focus", "focus")
p.defnoun("cross", "cross")
p.defnoun("success", "success")
p.defnoun("address", "address")
p.defnoun("pernicious", "pernicious")
p.defnoun("various", "various")
p.defnoun("continuous", "continuous")
p.defnoun("obvious", "obvious")
p.defnoun("process", "process")
p.defnoun("analysis", "analysis")
p.defnoun("enormous", "enormous")
p.defnoun("previous", "previous")
p.defnoun("homogeneous", "homogeneous")
p.defnoun("heterogeneous", "heterogeneous")
p.defnoun("tardiness", "tardiness")
p.defnoun("uniqueness", "uniqueness")
p.defnoun("fitness", "fitness")
p.defnoun("campus", "campus")
p.defnoun("relatedness", "relatedness")
p.defnoun("randomness", "randomness")
p.defnoun("express", "express")

p.defnoun("slackness", "slackness")
p.defnoun("simultaneous", "simultaneous")
p.defnoun("usefulness", "usefulness")
p.defnoun("synchronous", "synchronous")
p.defnoun("asynchronous", "asynchronous")
p.defnoun("hypothesis", "hypothesis")
p.defnoun("involves", "involves")
p.defnoun("witness", "witness")
p.defnoun("useless", "useless")
p.defnoun("correctness", "correctness")
p.defnoun("effectiveness", "effectiveness")
p.defnoun("exactness", "exactness")
p.defnoun("benders", "benders")
p.defnoun("references", "references")


#from utils import Config
import utils
import importlib
importlib.reload(utils)

<module 'utils' from '/home/mcaserta/research/nlp/matheur/utils.py'>

In [2]:
from nltk.corpus import stopwords
stop_words = stopwords.words("english")
stop_words += ['a', 'about', 'above', 'across', 'after', 'afterwards']
stop_words += ['again', 'against', 'all', 'almost', 'alone', 'along']
stop_words += ['already', 'also', 'although', 'always', 'am', 'among']
stop_words += ['amongst', 'amoungst', 'amount', 'an', 'and', 'another']
stop_words += ['any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere']
stop_words += ['are', 'around', 'as', 'at', 'back', 'be', 'became']
stop_words += ['because', 'become', 'becomes', 'becoming', 'been']
stop_words += ['before', 'beforehand', 'behind', 'being', 'below']
stop_words += ['beside', 'besides', 'between', 'beyond', 'bill', 'both']
stop_words += ['bottom', 'but', 'by', 'call', 'can', 'cannot', 'cant']
stop_words += ['co', 'computer', 'con', 'could', 'couldnt', 'cry', 'de']
stop_words += ['describe', 'detail', 'did', 'do', 'done', 'down', 'due']
stop_words += ['during', 'each', 'eg', 'eight', 'either', 'eleven', 'else']
stop_words += ['elsewhere', 'empty', 'enough', 'etc', 'even', 'ever']
stop_words += ['every', 'everyone', 'everything', 'everywhere', 'except']
stop_words += ['few', 'fifteen', 'fifty', 'fill', 'find', 'fire', 'first']
stop_words += ['five', 'for', 'former', 'formerly', 'forty', 'found']
stop_words += ['four', 'from', 'front', 'full', 'further', 'get', 'give']
stop_words += ['go', 'had', 'has', 'hasnt', 'have', 'he', 'hence', 'her']
stop_words += ['here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers']
stop_words += ['herself', 'him', 'himself', 'his', 'how', 'however']
stop_words += ['hundred', 'i', 'ie', 'if', 'in', 'inc', 'indeed']
stop_words += ['interest', 'into', 'is', 'it', 'its', 'itself', 'keep']
stop_words += ['last', 'latter', 'latterly', 'least', 'less', 'ltd', 'made']
stop_words += ['many', 'may', 'me', 'meanwhile', 'might', 'mill', 'mine']
stop_words += ['more', 'moreover', 'most', 'mostly', 'move', 'much']
stop_words += ['must', 'my', 'myself', 'name', 'namely', 'neither', 'never']
stop_words += ['nevertheless', 'next', 'nine', 'no', 'nobody', 'none']
stop_words += ['noone', 'nor', 'not', 'nothing', 'now', 'nowhere', 'of']
stop_words += ['off', 'often', 'on','once', 'one', 'only', 'onto', 'or']
stop_words += ['other', 'others', 'otherwise', 'our', 'ours', 'ourselves']
stop_words += ['out', 'over', 'own', 'part', 'per', 'perhaps', 'please']
stop_words += ['put', 'rather', 're', 's', 'same', 'see', 'seem', 'seemed']
stop_words += ['seeming', 'seems', 'serious', 'several', 'she', 'should']
stop_words += ['show', 'side', 'since', 'sincere', 'six', 'sixty', 'so']
stop_words += ['some', 'somehow', 'someone', 'something', 'sometime']
stop_words += ['sometimes', 'somewhere', 'still', 'such', 'system', 'take']
stop_words += ['ten', 'than', 'that', 'the', 'their', 'them', 'themselves']
stop_words += ['then', 'thence', 'there', 'thereafter', 'thereby']
stop_words += ['therefore', 'therein', 'thereupon', 'these', 'they']
stop_words += ['thick', 'thin', 'third', 'this', 'those', 'though', 'three']
stop_words += ['three', 'through', 'throughout', 'thru', 'thus', 'to']
stop_words += ['together', 'too', 'top', 'toward', 'towards', 'twelve']
stop_words += ['twenty', 'two', 'un', 'under', 'unless', 'until', 'up', 'upon']
stop_words += ['us', 'very', 'via', 'was', 'we', 'well', 'were', 'what']
stop_words += ['whatever', 'when', 'whence', 'whenever', 'where']
stop_words += ['whereafter', 'whereas', 'whereby', 'wherein', 'whereupon']
stop_words += ['wherever', 'whether', 'which', 'while', 'whither', 'who']
stop_words += ['whoever', 'whole', 'whom', 'whose', 'why', 'will', 'with']
stop_words += ['within', 'without', 'would', 'yet', 'you', 'your']
stop_words += ['yours', 'yourself', 'yourselves']

**NOTE**: All words in the dictionary are transformed in lower case!

In [3]:
fullpath = path.join(utils.Config.prefixDict,utils.Config.vocab_folder)
name_vocab = fullpath + "embed500.vocab"
#name_vocab = fullpath + "embed.vocab"

with open(name_vocab) as f:
    vocab_list = map(str.strip,f.readlines())
vocab_dict = {w.lower():k for k,w in enumerate(vocab_list)} # transform all words in lower case



In [4]:
ww = "nature-designed"
print("Test = '{0}' in dict ? {1}".format(ww, ww in vocab_dict))
print(vocab_dict["aa"])

Test = 'nature-designed' in dict ? True
155935


This is just a test. I want to verify whether some technical words are in the dictionary (which does not seem the case). In addition, I check whether the `lower()` operator has an effect on the entire word.

Below, we eliminate the "-" from hyphenation. I thought it was going to be more complex, to deal with cases like "state-of-the-art." However, I observed that, when the hypenation is joining multiple words, these are read and N-grams (a single word, e.g., "worst-case.") Thus, nothing needs to be done in these cases. 

What we need to fix is the hyphenation at the end of the line. The pdf file might separate words to deal with the end of line. In these cases, we just need to join two consecutive words. Thus, the while cycle below is actually never used, since precisely two "words" are neeeded to recompose the original word.

Here we have *a little issue*. Since the chunk of text we obtain from the pdf does not extend over multiple pages, it could happen that a word is split and separated over two lines belonging to different pages. For example, imagine that the first page ends with "mor-" and teh second page starts with "ning". In this case, since the chunk finishes with the last word of the page, we are not able to "look at the next word" and thus join "morn-" with whatever follows. Of course, this could be fixed, but I do not think it is so important. In the current version, we simply skip the last word with the hyphen, if it is the last word of a chunk.

In [5]:
def create_dict(fileToOpen, lower=True):
    with open(fileToOpen, "r") as ff:
        my_list = map(str.strip, ff.readlines())
    if lower:
        my_dict = {w.lower():k for k,w in enumerate(my_list)}
    else:
        my_dict = {w:k for k,w in enumerate(my_list)}

    return my_dict

def create_acrons_dict(fileToOpen):
    my_dict = dict()
    with open(fileToOpen, "r") as ff:
        for row in ff:
            #row = ff.readline()
            #print("row=", row)
            row = word_tokenize(row)
            my_dict[row[0]] = row[1]
    
    return my_dict

This chunk is used to load the list of N-grams built while preprocessing all the documents in the first run. The idea is to build such list of bi-grams and use them to transform disjoint words into bi-grams. For example, many people might say *worst case*, while others might use *worst-case*. We transform all the ocurrences into *worst-case*. In addition, it makes sense to transform some key sentences into n-grams. Some examples are:
* mathematical programming --> mathematical_programming
* data mining
* operations research
* management science
* corridor method
* tabu search
* genetic algorithm(s)

The idea is to avoid counting management and science separately. Rather, we want it to be treated as a unique word.


In [16]:
ngrams_dict  = create_dict(utils.Config.ngramsFile, True)
acrons_dict  = create_acrons_dict(utils.Config.acronsFile) # do not transform to lower case 
special_dict = create_acrons_dict(utils.Config.specialFile) # do not transform to lower case 

chunk = "mathematical primal-and-dual for lagrangian and Bound, and Lagrangian two branch-and-cut math-heuristics based on the model"
chunk = re.sub('[“”,.’´`{}()&%\[\]]','', chunk)
print("Chunk here =", chunk)
chunk = convert_special_words(chunk, special_dict)
print("Chunk after =", chunk)
chunk = change_acronyms(chunk, acrons_dict)
words = [w.lower() for w in word_tokenize(chunk) if w.lower() not in stop_words]
newWords = [w.split("-") for w in words]
newWords = list(itertools.chain.from_iterable(newWords))
chunk = ' '.join(newWords) # re-join words and substitute
print("Now chunk is ;; ", chunk)    

print("WORDS = ", words)
print("NEWWORDS = ", newWords)
words = include_ngrams(newWords, ngrams_dict)
#print("CHUNK = ", chunk)
print("WORDS = ", words)

Chunk here = mathematical primal-and-dual for lagrangian and Bound and Lagrangian two branch-and-cut math-heuristics based on the model
Chunk after = mathematical primal-dual for lagrangean and Bound and lagrangean two branch-cut matheuristic based on the model
Now chunk is ;;  mathematical primal dual lagrangean bound lagrangean branch cut matheuristic based model
WORDS =  ['mathematical', 'primal-dual', 'lagrangean', 'bound', 'lagrangean', 'branch-cut', 'matheuristic', 'based', 'model']
NEWWORDS =  ['mathematical', 'primal', 'dual', 'lagrangean', 'bound', 'lagrangean', 'branch', 'cut', 'matheuristic', 'based', 'model']
WORDS =  ['mathematical', 'primal-dual', 'lagrangean', 'bound', 'lagrangean', 'branch-cut', 'matheuristic', 'based', 'model']


In [6]:
def include_ngrams(newWords, ngrams_dict):
    newWords = [w.split("-") for w in newWords]
    newWords = list(itertools.chain.from_iterable(newWords))
    chunk = ' '.join(newWords) # re-join words and substitute
    # look for n-grams
    for ngram in ngrams_dict:
        #print("Searching for ", ngram)
        #ngram = re.sub('-', ' ', ngram)
        #print("... in chunk ", chunk)
        auxChunk = re.sub('-', ' ', chunk) # to address cases as iterated local-search
        block = re.compile(ngram, re.IGNORECASE).finditer(auxChunk)

        init   = []
        ending = []
        for m in block:
            init.append(m.start())
            ending.append(m.end())
        #print("init and end ", init, ending)
        if len(init) > 0:
            #print("Changing ", ngram)
            i = 0
            sentence = chunk[:init[0]]
            for i in range(len(init)):
                sentence += re.sub(' ','-',chunk[init[i]:ending[i]])
                if i < len(init) -1 :
                    sentence += chunk[ending[i]:init[i+1]]
                else:
                    sentence += chunk[ending[i]:]
            chunk = sentence
            newWords = [w for w in word_tokenize(sentence)]
            #print("Now we have ", newWords)
            #print("** N-gram found : ", ngram)
            #input("aka")
    
    return newWords
          
def change_acronyms(chunk, acrons_dict):
    for acron in acrons_dict:
        re.sub('-',' ',acrons_dict[acron])
        chunk = re.sub(' ' + acron + ' ', ' ' + acrons_dict[acron] + ' ', chunk)
    return chunk

def convert_special_words(chunk, special_dict):
    for special in special_dict:
        chunk = re.sub(special, special_dict[special], chunk, flags=re.I) #ignore case
    return chunk
        

# list of fixes due to OCR errors
OCRFixes = {'ﬁ'  : 'fi',
            'ﬀ'  : 'ff',
            'ﬂ'  : 'fl',
            'ﬃ' : 'ffi'
}
replacementDictRegex = re.compile("(%s)" % "|".join(map(re.escape, OCRFixes.keys())))


def fix_OCR_errors(chunk):
    #chunk = re.sub('ﬁ','fi', chunk) 
    chunk = replacementDictRegex.sub(lambda mo:OCRFixes.get(mo.group(1),mo.group(1)), chunk)
    return chunk


def process_chunk(chunk, ngrams_dict, acrons_dict, special_dict):

    chunk = fix_OCR_errors(chunk)
    chunk = re.sub('[“”,.’´`{}()&%\[\]]','', chunk)
    chunk = re.sub('/', ' ', chunk)
    chunk = convert_special_words(chunk, special_dict)
    chunk = change_acronyms(chunk, acrons_dict)

    words = [w.lower() for w in word_tokenize(chunk) if w.lower() not in stop_words]
    newWords = []
    i = 0
    while i < len(words): # skip last element (we could lose a few words here)
        w = words[i]
        if len(w) <= 1:
            i += 1
            continue
        if i < len(words)-1:
            concat = ""
            if w[-1] == "-":
                concat += w[:-1]
                i += 1
                concat += words[i] # go to next word and join
                #print("** Concatenated word = ", concat)
                w = concat
        
        newWords.append(w)
        i += 1
        
    # convert nouns to singular
    newWords = [p.singular_noun(w) if p.singular_noun(w) else w for w in newWords]

    # eliminate all bi-grams (we recreated them according to our list)
    if len(newWords) > 0:
        newWords = include_ngrams(newWords, ngrams_dict)
        
    newWords = add_chunk_to_corpus(newWords)
    return newWords


def add_chunk_to_corpus(words):
    corrected = []  
    toAdd = False
    for w in words:    
        if (len(re.findall("-",w)) > 0 or (w.isalpha()) and len(w) > 2 and w in vocab_dict): # eliminate short words
#            if w not in vocab_dict:
#                print("?? {0} not in dict".format(w))
#                toAdd = True
            # if it is a composed word, e.g., "worst-case" or is alphabetical (note that "worst-case" is NOT alpha)
#            if w not in vocab_dict:
#                print("Word : ",w, "not in dictionary")

            corrected.append(w)
#    if toAdd:
#        input("check DICT")
    return corrected

In [7]:
def process_doc_info(doc):
    info = doc.info[0]
    #print(info)
    author = title = kw = ''
    if 'Author' in info.keys():
        #author = info["Author"].decode('utf-8')
        author = info["Author"].decode('iso-8859-1')
        #author = info["Author"]
    if 'Title' in info.keys():
        title  = info["Title"].decode('iso-8859-1')
    if 'Keywords' in info.keys():
        kw     = info["Keywords"].decode('iso-8859-1')
    return author, title, kw

# Main Cycle

Open a pdf file and process it. Check the following:
* proper elimination of hyphens at the end of the line
* proper dealing with n-grams
* generation of `myword.txt`, my dictionary of n-grams. This will be deactivated after a full first run

In reality, what we need to do is to re-read all the documents and introduce the n-grams (e.g., tabu_search, etc.)  We also need to deal with abbreviations, e.g.:
* GA --> genetic_algorithm
* TS --> tabu_search
* CM --> corridor_method

## Generate Mapping

In [43]:
year0   = "2001"
yearT   = "2019"
yearSet = list(range(int(year0), int(yearT)+1))

#df        = pd.DataFrame(columns=['id', 'year', 'name', 'nrWords', 'author', 'title', 'kw'])
df        = pd.DataFrame(columns=['id', 'year', 'name', 'nrWords'])
counter   = 0
for year in yearSet:
    print("** Year = ", year, "**", counter)
    listOfFiles = glob.glob(path.join(utils.Config.prefix,utils.Config.pdf_dir,str(year),"*.pdf"))

    for pdfFile in listOfFiles:

        #print("Opening FILE = ", pdfFile)
        filename = os.path.basename(pdfFile)
        name, file_extension = os.path.splitext(filename)
        filename = name + ".p"
        fullFileName = path.join(utils.Config.prefix, utils.Config.p_dir,str(year),filename)
        if os.path.isfile(fullFileName):
            aa = pickle.load(open(fullFileName, 'rb'))
            nrWords = len(aa)
            #document = open(pdfFile, 'rb')
            #parser   = PDFParser(document)
            #doc      = PDFDocument(parser)
            #author, title, kw = process_doc_info(doc)
            #df.loc[counter] = [counter, year, name, nrWords, author, title, kw]  
            df.loc[counter] = [counter, year, name, nrWords]   
            counter += 1
        else:
            print("File {0} does not exist.".format(fullFileName))

if not df.empty:
    df.to_csv(utils.Config.mappingFile)
print("Mapping saved as '{0}'. Total of {1} files.".format(utils.Config.mappingFile, counter))

** Year =  2001 ** 0
** Year =  2002 ** 2
** Year =  2003 ** 2
** Year =  2004 ** 4
** Year =  2005 ** 5
** Year =  2006 ** 7
** Year =  2007 ** 9
** Year =  2008 ** 11
** Year =  2009 ** 12
** Year =  2010 ** 17
** Year =  2011 ** 21
** Year =  2012 ** 24
** Year =  2013 ** 24
** Year =  2014 ** 33
** Year =  2015 ** 52
** Year =  2016 ** 68
** Year =  2017 ** 84
** Year =  2018 ** 108
** Year =  2019 ** 157
Mapping saved as 'mapping.csv'. Total of 190 files.


## Process Documents

### My Matheuristic Papers

In [42]:
if os.path.exists(os.path.dirname(utils.Config.mappingFile)):
    dfMap        = pd.read_csv(utils.Config.mappingFile)
    idCounter = dfMap["id"].max() + 1
else:
    idCounter = 0
    
#biblioTitle  = ["Bibliography", "References", "Acknowledgements"]
ngrams_dict  = create_dict(utils.Config.ngramsFile, True)
acrons_dict  = create_acrons_dict(utils.Config.acronsFile) # do not transform to lower case 
special_dict = create_acrons_dict(utils.Config.specialFile) 

year0   = "2001"
yearT   = "2019"
yearSet = list(range(int(year0), int(yearT)+1))

counter   = 0
for year in yearSet:
    listOfFiles = glob.glob(path.join(utils.Config.prefix,utils.Config.pdf_dir,str(year),"*.pdf"))
    
    for pdfFile in listOfFiles:

        #print("Opening FILE = ", pdfFile)
        filename = os.path.basename(pdfFile)
        filename, file_extension = os.path.splitext(filename)
        filename += ".p"
        fullFileName = path.join(utils.Config.prefix, utils.Config.p_dir,str(year),filename)
        #if filename == "taillard-01.p" or filename == "hansen-01.p" or filename == "dumitrescu-03.p":
        #    continue
        if os.path.isfile(fullFileName):
        #if filename != "ren-14.p":
            print("... Processed file {0} already exists in folder '{1}'. Skip this.".format(filename, path.join(utils.Config.p_dir, str(year))))
            #continue
        #input("aka")
        else:
            print("+++ Processing file {0}. Destination folder ='{1}'.".format(filename, path.join(utils.Config.p_dir, str(year))))

        with open(pdfFile, 'rb') as ff: # I am not able to find a better way to get nr pages
            pdfAux = pyPdf.PdfFileReader(ff)
            totPages = pdfAux.getNumPages()
        
        document = open(pdfFile, 'rb')
        
        rsrcmgr     = PDFResourceManager() #Create resource manager
        laparams    = LAParams() # Set parameters for analysis.
        device      = PDFPageAggregator(rsrcmgr, laparams=laparams) # Create a PDF page aggregator object.
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        
        i        = 0
        doc      = []
        biblio   = False
        for page in PDFPage.get_pages(document):
            i += 1 # update page number in pdf
            #print("Processing page ", i)
            percentPages = i/int(totPages)
            
            interpreter.process_page(page)
            layout = device.get_result() # receive the LTPage object for the page.
            for element in layout:
                if isinstance(element, LTTextBoxHorizontal):
                    el = element.get_text()
 
                    # next, attempt to skip bibliography
                    #if len(words) <= 5 and re.compile('|'.join(utils.Config.biblioTitle)).search(el):
                    if percentPages > 0.7 and re.compile('|'.join(utils.Config.biblioTitle),re.I).search(el):
                        print("Ref on page ", i, " = ", el)                        
                        biblio = True
                        break
                        
                    #print("\nBefore", el)
                    words = process_chunk(el, ngrams_dict, acrons_dict, special_dict)
                    #print("After", words)

                    for w in words:
                        doc.append(w)
            if biblio:
                break
        if not os.path.exists(os.path.dirname(fullFileName)):
            os.makedirs(os.path.dirname(fullFileName))
        print("Saving file ", filename)
        print("*"*50,"\n")
        pickle.dump(doc,open(fullFileName, 'wb'))

        counter   += 1

print("Done with processing phase: Processed {0} files.".format(counter))

... Processed file taillard-01.p already exists in folder 'data/p/2001'. Skip this.
Ref on page  15  =  References

Saving file  taillard-01.p
************************************************** 

... Processed file hansen-01.p already exists in folder 'data/p/2001'. Skip this.


Ref on page  15  =  References

Saving file  hansen-01.p
************************************************** 

... Processed file dumitrescu-03.p already exists in folder 'data/p/2003'. Skip this.
Ref on page  27  =  Acknowledgements

Saving file  dumitrescu-03.p
************************************************** 

... Processed file fischetti-03.p already exists in folder 'data/p/2003'. Skip this.
Ref on page  24  =  Acknowledgements. Work partially supported by MIUR and CNR, Italy, and by the EU project “DONET”.
Thanks are due to anonymous referees for detailed and helpful comments.

Saving file  fischetti-03.p
************************************************** 

... Processed file fischetti-04.p already exists in folder 'data/p/2004'. Skip this.
Ref on page  17  =  References

Saving file  fischetti-04.p
************************************************** 

... Processed file danna-05.p already exists in folder 'data/p/2005'. Skip this.
Ref on page  19  =  Acknowledgements. We wish to

Ref on page  10  =  References

Saving file  lin-14.p
************************************************** 

... Processed file brouer-14.p already exists in folder 'data/p/2014'. Skip this.
Ref on page  17  =  Acknowledgments

Saving file  brouer-14.p
************************************************** 

... Processed file wang-17.p already exists in folder 'data/p/2014'. Skip this.
Ref on page  14  =  Acknowledgments

Saving file  wang-17.p
************************************************** 

... Processed file caserta-14.p already exists in folder 'data/p/2014'. Skip this.
Ref on page  12  =  References

Saving file  caserta-14.p
************************************************** 

... Processed file cafieri-14.p already exists in folder 'data/p/2014'. Skip this.
Ref on page  7  =  References

Saving file  cafieri-14.p
************************************************** 

... Processed file archetti-14.p already exists in folder 'data/p/2014'. Skip this.
Ref on page  21  =  Acknowledgme

Ref on page  16  =  Acknowledgments This work has been partially funded by the Spanish Ministry of Economy and Com-
petitiveness (project TIN2012-32608). Eduardo Lalla-Ruiz thanks the Canary Government for the financial
support he receives through his doctoral grant. An earlier version of this paper was presented at LION8 [13].

Saving file  lallaRuiz-16.p
************************************************** 

... Processed file lusby-16.p already exists in folder 'data/p/2016'. Skip this.
Ref on page  12  =  References 

Saving file  lusby-16.p
************************************************** 

... Processed file ritzinger-16.p already exists in folder 'data/p/2016'. Skip this.
Ref on page  17  =  Acknowledgments We would like to thank the anonymous referees for their invaluable remarks, helping to
improve our paper. We would also like to thank Gerhard Hiermann for many insightful discussions. This work
was partially funded by the Austrian Federal Ministry for Transport, Innovation an

Ref on page  17  =  References 

Saving file  akbari-17.p
************************************************** 

... Processed file grangier-17.p already exists in folder 'data/p/2017'. Skip this.
Ref on page  10  =  Acknowledgements 

Saving file  grangier-17.p
************************************************** 

... Processed file martinez-gavara-17.p already exists in folder 'data/p/2017'. Skip this.
Ref on page  14  =  Acknowledgments 

Saving file  martinez-gavara-17.p
************************************************** 

... Processed file parragh-17.p already exists in folder 'data/p/2017'. Skip this.
Ref on page  17  =  References 

Saving file  parragh-17.p
************************************************** 

... Processed file gendron-18.p already exists in folder 'data/p/2018'. Skip this.
Ref on page  12  =  Acknowledgments 

Saving file  gendron-18.p
************************************************** 

... Processed file summary.p already exists in folder 'data/p/2018'. Skip t

Ref on page  10  =  Acknowledgments 

Saving file  calvete-18.p
************************************************** 

... Processed file wu-18.p already exists in folder 'data/p/2018'. Skip this.
Ref on page  12  =  Acknowledgments 

Saving file  wu-18.p
************************************************** 

... Processed file paiva-18.p already exists in folder 'data/p/2018'. Skip this.
Ref on page  21  =  Acknowledgements 

Saving file  paiva-18.p
************************************************** 

... Processed file keshin-18.p already exists in folder 'data/p/2018'. Skip this.
Ref on page  14  =  Acknowledgments 

Saving file  keshin-18.p
************************************************** 

... Processed file ghaffarinasab-18.p already exists in folder 'data/p/2018'. Skip this.
Ref on page  16  =  Acknowledgments 

Saving file  ghaffarinasab-18.p
************************************************** 

... Processed file framinan-18.p already exists in folder 'data/p/2018'. Skip this.
Re

Ref on page  14  =  References 

Saving file  brech-19.p
************************************************** 

... Processed file hernandez-19.p already exists in folder 'data/p/2019'. Skip this.
Ref on page  30  =  Acknowledgements The authors gratefully acknowledge funding provided by the Canadian Natural Sci-
ences and Engineering Research Council. The authors thank the two anonymous referees for their insightful
comments and suggestions that helped improve the content and the presentation of the paper.

Saving file  hernandez-19.p
************************************************** 

... Processed file bertazzi-19.p already exists in folder 'data/p/2019'. Skip this.
Ref on page  20  =  with a homogeneous fleet of vehicles is studied. While classical IRPs have been studied extensively, the
multi-depot case represents a variant not well investigated despite its applications in last-mile delivery optimization, when the
Vendor-Managed Inventory paradigm is applied. A MILP formulation is 

### From Arxiv.org

Use this to process files from arxiv.org.

In [64]:
db = pickle.load(open(utils.Config.db_dir, "rb"))
df = pd.DataFrame(columns=['id', 'year', 'name', 'nrWords', 'title', 'author'])
counter = 1000
for item in db:
    #print(db[item])
    name = os.path.basename(db[item]["id"])
    ff = path.join(utils.Config.pArxiv_dir, name+".p")
    if os.path.isfile(ff):
        #print("Mapping file ", ff)
        doc = pickle.load(open(ff, 'rb'))
        nrWords = len(doc)
        df.loc[counter] = [counter, db[item]["published_parsed"][0], name, nrWords, db[item]["title"], db[item]["author"]]

        counter += 1
print(df)
df.to_csv(utils.Config.mappingArxiv) 
print("ArXiv map saved in file '{0}'.".format(utils.Config.mappingArxiv))

        id  year          name nrWords  \
1000  1000  2010   1004.4170v1    1473   
1001  1001  2018  1804.09813v2    3957   
1002  1002  2012   1207.1794v1   18000   
1003  1003  2011   1106.2686v1    1941   
1004  1004  2014   1407.4364v1    1711   
1005  1005  2017  1711.09369v1     362   
1006  1006  2013   1303.2215v1    2728   
1007  1007  2018  1807.06731v1    5237   
1008  1008  2012   1206.1579v2    1988   
1009  1009  2018  1802.06318v1    4602   
1010  1010  2015  1504.02590v1     958   
1011  1011  2017  1701.08709v2    2819   
1012  1012  2012   1211.6409v1    1539   
1013  1013  2019  1903.02079v1    1381   
1014  1014  2018  1806.01681v1    2604   
1015  1015  2015  1511.00195v1    2488   
1016  1016  2018  1810.04348v2    5561   
1017  1017  2014   1404.6696v1    2679   
1018  1018  2012   1207.4631v1    2416   
1019  1019  2015  1506.00211v1    2667   
1020  1020  2014   1401.3794v2    3715   
1021  1021  2018  1804.02851v2    4689   
1022  1022  2018  1802.08729v1    

In [65]:
df.to_csv(utils.Config.mappingArxiv)

In [ ]:
if os.path.exists(os.path.dirname(utils.Config.mappingArxiv)):
    dfMap        = pd.read_csv(utils.Config.mappingArxiv)
    idCounter = dfMap["id"].max() + 1
else:
    idCounter = 0
    
ngrams_dict  = create_dict(utils.Config.ngramsFile, True)
acrons_dict  = create_acrons_dict(utils.Config.acronsFile) # do not transform to lower case 
special_dict = create_acrons_dict(utils.Config.specialFile) 

counter   = 0

listOfFiles = glob.glob(path.join(utils.Config.prefix,utils.Config.arxiv_dir,"*.pdf"))
#listOfFiles = ['/home/mcaserta/research/nlp/matheur/data/pdf/arxiv/1012.5815v2.pdf']
for pdfFile in listOfFiles:

    #print("Opening FILE = ", pdfFile)
    filename = os.path.basename(pdfFile)
    filename, file_extension = os.path.splitext(filename)
    filename += ".p"
    fullFileName = path.join(utils.Config.prefix, utils.Config.pArxiv_dir,filename)
    if os.path.isfile(fullFileName):
    #if filename != "ren-14.p":
        print("... Processed file {0} already exists in folder '{1}'. Skip this.".format(filename, path.join(utils.Config.pArxiv_dir)))
        #continue
    #input("aka")
    else:
        print("+++ Processing file {0}. Destination folder ='{1}'.".format(filename, path.join(utils.Config.pArxiv_dir)))

    with open(pdfFile, 'rb') as ff: # I am not able to find a better way to get nr pages
        pdfAux = pyPdf.PdfFileReader(ff)
        totPages = pdfAux.getNumPages()

    document = open(pdfFile, 'rb')

    rsrcmgr     = PDFResourceManager() #Create resource manager
    laparams    = LAParams() # Set parameters for analysis.
    device      = PDFPageAggregator(rsrcmgr, laparams=laparams) # Create a PDF page aggregator object.
    interpreter = PDFPageInterpreter(rsrcmgr, device)

    i        = 0
    doc      = []
    biblio   = False
    for page in PDFPage.get_pages(document):
        i += 1 # update page number in pdf
        #print("Processing page ", i)
        percentPages = i/int(totPages)

        interpreter.process_page(page)
        layout = device.get_result() # receive the LTPage object for the page.
        for element in layout:
            if isinstance(element, LTTextBoxHorizontal):
                el = element.get_text()

                # next, attempt to skip bibliography
                #if len(words) <= 5 and re.compile('|'.join(utils.Config.biblioTitle)).search(el):
                if percentPages > 0.5 and re.compile('|'.join(utils.Config.biblioTitle),re.I).search(el):
                    print("Ref on page ", i, " = ", el)                        
                    biblio = True
                    break

                #print("\nBefore", el)
                words = process_chunk(el, ngrams_dict, acrons_dict, special_dict)
                #print("After", words)

                for w in words:
                    doc.append(w)
        if biblio:
            break
    if not os.path.exists(os.path.dirname(fullFileName)):
        os.makedirs(os.path.dirname(fullFileName))
    print("Saving file ", filename)
    print("*"*50,"\n")
    pickle.dump(doc,open(fullFileName, 'wb'))

    counter   += 1
    #input("aka")

print("Done with processing phase: Processed {0} files.".format(counter))

... Processed file 1303.1271v5.p already exists in folder 'data/pArxiv'. Skip this.
Ref on page  33  =  Acknowledgments

Saving file  1303.1271v5.p
************************************************** 

... Processed file 1611.08775v1.p already exists in folder 'data/pArxiv'. Skip this.
Ref on page  9  =  REFERENCES 

Saving file  1611.08775v1.p
************************************************** 

... Processed file 1303.6145v2.p already exists in folder 'data/pArxiv'. Skip this.
Ref on page  12  =  References

Saving file  1303.6145v2.p
************************************************** 

... Processed file 1401.3492v1.p already exists in folder 'data/pArxiv'. Skip this.
Ref on page  37  =  Acknowledgments
We thank Kevin Murphy for many helpful discussions regarding this work. We also thank Domagoj
Babi´c, the author of SPEAR, and Dave Tompkins, the author of the UBCSAT SAPS implementation
we used in our experiments. We thank the researchers who provided the instances or instance
genera

Saving file  1406.7473v1.p
************************************************** 

... Processed file 1704.00853v1.p already exists in folder 'data/pArxiv'. Skip this.
Ref on page  19  =  Table 1: Problem domains and key references

Saving file  1704.00853v1.p
************************************************** 

... Processed file 1303.3154v3.p already exists in folder 'data/pArxiv'. Skip this.
Ref on page  11  =  REFERENCES

Saving file  1303.3154v3.p
************************************************** 

... Processed file 0809.0961v1.p already exists in folder 'data/pArxiv'. Skip this.
Ref on page  6  =  Acknowledgements

Saving file  0809.0961v1.p
************************************************** 

... Processed file 1401.7074v1.p already exists in folder 'data/pArxiv'. Skip this.
Ref on page  5  =  REFERENCES

Saving file  1401.7074v1.p
************************************************** 

... Processed file 1809.04942v1.p already exists in folder 'data/pArxiv'. Skip this.
Ref on page 

In [23]:

el = "REFERENCES" 
print(re.compile('|'.join(utils.Config.biblioTitle), re.I).search(el))


<_sre.SRE_Match object; span=(0, 10), match='REFERENCES'>


Import mapping file to switch from name to id and viceversa.

In [8]:
def createMap(mappingFile):
    dfMap = pd.read_csv(mappingFile)
    #print(dfMap)
    file2tag = {f:t for f,t in zip(dfMap["name"],dfMap["id"])}
    tag2file = {t:f for t,f in zip(dfMap["id"],dfMap["name"])}
    print("Legths :: From file to TAG = {0}; from TAG to file = {1}".format(len(file2tag), len(tag2file)))
    #print("Name of file with id = {0} is {1}".format(1,tag2file[1]))
    #print("ID of file with name = {0} is {1}".format("puchinger-05", file2tag["puchinger-05"]))
    
    return file2tag, tag2file

In [22]:
def wordListToFreqDict(wordlist):
    wordfreq = [wordlist.count(p) for p in wordlist]
    return dict(zip(wordlist,wordfreq))

def sortFreqDict(freqdict):
    aux = [(freqdict[key], key) for key in freqdict]
    aux.sort()
    aux.reverse()
    return aux

# Doc2vec Creation

In [12]:
file2tag, tag2file = createMap(utils.Config.mappingFile)
analyzedDocument = namedtuple('AnalyzedDocument','words tags')
year0 = "2001"
yearT = "2019"
yearSet = list(range(int(year0), int(yearT)+1))

corpus = []
counter  = 0
for year in yearSet:
    print("** Year = ", year, "**", counter)

    listOfFiles = glob.glob(path.join(utils.Config.prefix,utils.Config.p_dir,str(year),"*.p"))

    for pFile in listOfFiles:

        #print("Opening FILE = ", pFile)
        filename = os.path.basename(pFile)
        name, file_extension = os.path.splitext(filename)
        if os.path.isfile(pFile):
            ff = path.join(utils.Config.p_dir, str(year), filename)
            #print("Processing file '{0}'".format(ff))
            doc = pickle.load(open(ff, 'rb'))
            tags = [str(file2tag[name])]
            #print("... TAG = ", tags, " :: document length = ", len(doc), "words.")
            corpus.append(analyzedDocument(doc,tags))
            counter += 1
        else:
            print("** NOTE: File {0} not found!".format(filename))

# now add arXiv files
totMine = counter
file2tagArxiv, tag2fileArxiv = createMap(utils.Config.mappingArxiv)
listOfFiles = glob.glob(path.join(utils.Config.prefix,utils.Config.pArxiv_dir,"*.p"))

for pFile in listOfFiles:

    #print("Opening FILE = ", pFile)
    filename = os.path.basename(pFile)
    name, file_extension = os.path.splitext(filename)
    if os.path.isfile(pFile):
        ff = path.join(utils.Config.pArxiv_dir, filename)
        print("Processing file '{0}'".format(ff), "with name", name)
        doc = pickle.load(open(ff, 'rb'))
        tags = [str(file2tagArxiv[name])]
        #print("... TAG = ", tags, " :: document length = ", len(doc), "words.")
        corpus.append(analyzedDocument(doc,tags))
        counter += 1
    else:
        print("** NOTE: File {0} not found!".format(ff))
    #input("aka")
print("Loaded corpus with {0} documents [Mine ({1}) + Arxiv ({2})].".format(len(corpus), totMine, len(corpus)-totMine))

Legths :: From file to TAG = 190; from TAG to file = 190
** Year =  2001 ** 0
** Year =  2002 ** 2
** Year =  2003 ** 2
** Year =  2004 ** 4
** Year =  2005 ** 5
** Year =  2006 ** 7
** Year =  2007 ** 9
** Year =  2008 ** 11
** Year =  2009 ** 12
** Year =  2010 ** 17
** Year =  2011 ** 21
** Year =  2012 ** 24
** Year =  2013 ** 24
** Year =  2014 ** 33
** Year =  2015 ** 52
** Year =  2016 ** 68
** Year =  2017 ** 84
** Year =  2018 ** 108
** Year =  2019 ** 157
Legths :: From file to TAG = 283; from TAG to file = 283
Processing file 'data/pArxiv/1208.0527v1.p' with name 1208.0527v1
Processing file 'data/pArxiv/0803.1728v1.p' with name 0803.1728v1
Processing file 'data/pArxiv/1212.5095v1.p' with name 1212.5095v1
Processing file 'data/pArxiv/1804.05429v1.p' with name 1804.05429v1
Processing file 'data/pArxiv/1003.0221v1.p' with name 1003.0221v1
Processing file 'data/pArxiv/1207.1794v1.p' with name 1207.1794v1
Processing file 'data/pArxiv/0708.0505v1.p' with name 0708.0505v1
Processin

Processing file 'data/pArxiv/0907.2990v1.p' with name 0907.2990v1
Processing file 'data/pArxiv/1312.5045v1.p' with name 1312.5045v1
Processing file 'data/pArxiv/1702.05603v3.p' with name 1702.05603v3
Processing file 'data/pArxiv/1211.6663v1.p' with name 1211.6663v1
Processing file 'data/pArxiv/1408.5348v1.p' with name 1408.5348v1
Processing file 'data/pArxiv/1303.2215v1.p' with name 1303.2215v1
Processing file 'data/pArxiv/1804.02851v2.p' with name 1804.02851v2
Processing file 'data/pArxiv/1003.1409v1.p' with name 1003.1409v1
Processing file 'data/pArxiv/1305.1880v1.p' with name 1305.1880v1
Processing file 'data/pArxiv/1502.02127v2.p' with name 1502.02127v2
Processing file 'data/pArxiv/1610.06009v1.p' with name 1610.06009v1
Processing file 'data/pArxiv/1305.3794v2.p' with name 1305.3794v2
Processing file 'data/pArxiv/1212.0220v1.p' with name 1212.0220v1
Processing file 'data/pArxiv/1605.05097v1.p' with name 1605.05097v1
Processing file 'data/pArxiv/1804.09813v2.p' with name 1804.09813v

Processing file 'data/pArxiv/1006.2806v1.p' with name 1006.2806v1
Processing file 'data/pArxiv/1607.04324v1.p' with name 1607.04324v1
Processing file 'data/pArxiv/1505.01742v1.p' with name 1505.01742v1
Processing file 'data/pArxiv/1309.5110v1.p' with name 1309.5110v1
Processing file 'data/pArxiv/0712.0451v1.p' with name 0712.0451v1
Processing file 'data/pArxiv/1811.01945v2.p' with name 1811.01945v2
Processing file 'data/pArxiv/1105.5900v2.p' with name 1105.5900v2
Processing file 'data/pArxiv/1903.02915v1.p' with name 1903.02915v1
Processing file 'data/pArxiv/1902.00107v1.p' with name 1902.00107v1
Processing file 'data/pArxiv/1801.03419v1.p' with name 1801.03419v1
Processing file 'data/pArxiv/1609.04069v1.p' with name 1609.04069v1
Processing file 'data/pArxiv/1109.3094v1.p' with name 1109.3094v1
Processing file 'data/pArxiv/1407.4364v1.p' with name 1407.4364v1
Processing file 'data/pArxiv/1602.07884v1.p' with name 1602.07884v1
Processing file 'data/pArxiv/1501.05354v2.p' with name 1501.

In [10]:
from gensim.models import doc2vec
import logging
logging.basicConfig(
        format='%(asctime)s : %(threadName)s : %(levelname)s : %(message)s',
        level=logging.INFO
)


In [13]:
#### creating doc2vec model for the current year
model = doc2vec.Doc2Vec(vector_size=300, window=100, min_count=5, epochs=250, workers=4, dm=1, negative=10)
model.build_vocab(corpus)
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)
model.init_sims(replace=True)

# save model on disk
fullname = path.join(utils.Config.models_dir,utils.Config.nameModel)
print("Saving model ", fullname)
model.save(fullname)
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

2019-03-11 14:08:19,141 : MainThread : INFO : collecting all words and their counts
2019-03-11 14:08:19,142 : MainThread : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-03-11 14:08:19,558 : MainThread : INFO : collected 21183 word types and 473 unique tags from a corpus of 473 examples and 1542379 words
2019-03-11 14:08:19,560 : MainThread : INFO : Loading a fresh vocabulary
2019-03-11 14:08:19,629 : MainThread : INFO : min_count=5 retains 9824 unique words (46% of original 21183, drops 11359)
2019-03-11 14:08:19,631 : MainThread : INFO : min_count=5 leaves 1522115 word corpus (98% of original 1542379, drops 20264)
2019-03-11 14:08:19,672 : MainThread : INFO : deleting the raw counts dictionary of 21183 items
2019-03-11 14:08:19,674 : MainThread : INFO : sample=0.001 downsamples 35 most-common words
2019-03-11 14:08:19,676 : MainThread : INFO : downsampling leaves estimated 1409913 word corpus (92.6% of prior 1522115)
2019-03-11 14:08:19,736 : MainT

2019-03-11 14:09:02,161 : MainThread : INFO : EPOCH 4 - PROGRESS: at 100.00% examples, 133229 words/s, in_qsize 0, out_qsize 1
2019-03-11 14:09:02,162 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-03-11 14:09:02,163 : MainThread : INFO : EPOCH - 4 : training on 1542379 raw words (1397913 effective words) took 10.5s, 133200 effective words/s
2019-03-11 14:09:03,181 : MainThread : INFO : EPOCH 5 - PROGRESS: at 7.61% examples, 120690 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:09:04,233 : MainThread : INFO : EPOCH 5 - PROGRESS: at 17.34% examples, 124613 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:09:05,245 : MainThread : INFO : EPOCH 5 - PROGRESS: at 24.74% examples, 125597 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:09:06,288 : MainThread : INFO : EPOCH 5 - PROGRESS: at 32.35% examples, 128393 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:09:07,399 : MainThread : INFO : EPOCH 5 - PROGRESS: at 39.96% examples, 128069 words/s, in_qsize 8, o

2019-03-11 14:09:52,502 : MainThread : INFO : EPOCH 9 - PROGRESS: at 77.80% examples, 132871 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:09:53,506 : MainThread : INFO : EPOCH 9 - PROGRESS: at 88.37% examples, 133085 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:09:54,427 : MainThread : INFO : worker thread finished; awaiting finish of 3 more threads
2019-03-11 14:09:54,441 : MainThread : INFO : worker thread finished; awaiting finish of 2 more threads
2019-03-11 14:09:54,458 : MainThread : INFO : worker thread finished; awaiting finish of 1 more threads
2019-03-11 14:09:54,489 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-03-11 14:09:54,490 : MainThread : INFO : EPOCH - 9 : training on 1542379 raw words (1398103 effective words) took 10.4s, 135025 effective words/s
2019-03-11 14:09:55,596 : MainThread : INFO : EPOCH 10 - PROGRESS: at 7.82% examples, 115597 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:09:56,625 : MainThread : INFO : EPOCH 10 

2019-03-11 14:10:40,175 : MainThread : INFO : EPOCH 14 - PROGRESS: at 32.98% examples, 129303 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:10:41,215 : MainThread : INFO : EPOCH 14 - PROGRESS: at 40.38% examples, 129840 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:10:42,227 : MainThread : INFO : EPOCH 14 - PROGRESS: at 51.80% examples, 130832 words/s, in_qsize 5, out_qsize 2
2019-03-11 14:10:43,309 : MainThread : INFO : EPOCH 14 - PROGRESS: at 63.00% examples, 131229 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:10:44,369 : MainThread : INFO : EPOCH 14 - PROGRESS: at 77.59% examples, 132350 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:10:45,373 : MainThread : INFO : EPOCH 14 - PROGRESS: at 88.79% examples, 133719 words/s, in_qsize 8, out_qsize 0
2019-03-11 14:10:46,271 : MainThread : INFO : worker thread finished; awaiting finish of 3 more threads
2019-03-11 14:10:46,301 : MainThread : INFO : worker thread finished; awaiting finish of 2 more threads
2019-03-11 14:10:46,302 : Main

2019-03-11 14:11:29,000 : MainThread : INFO : EPOCH 19 - PROGRESS: at 7.82% examples, 116441 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:11:30,009 : MainThread : INFO : EPOCH 19 - PROGRESS: at 17.55% examples, 125578 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:11:31,055 : MainThread : INFO : EPOCH 19 - PROGRESS: at 26.22% examples, 131102 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:11:32,073 : MainThread : INFO : EPOCH 19 - PROGRESS: at 33.62% examples, 132493 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:11:33,076 : MainThread : INFO : EPOCH 19 - PROGRESS: at 40.59% examples, 132554 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:11:34,129 : MainThread : INFO : EPOCH 19 - PROGRESS: at 53.91% examples, 134494 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:11:35,135 : MainThread : INFO : EPOCH 19 - PROGRESS: at 64.90% examples, 135199 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:11:36,157 : MainThread : INFO : EPOCH 19 - PROGRESS: at 77.80% examples, 134619 words/s, in_qsize 

...

2019-03-11 14:48:50,260 : MainThread : INFO : EPOCH 239 - PROGRESS: at 55.18% examples, 135984 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:48:51,280 : MainThread : INFO : EPOCH 239 - PROGRESS: at 66.17% examples, 135129 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:48:52,295 : MainThread : INFO : EPOCH 239 - PROGRESS: at 80.55% examples, 136280 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:48:53,305 : MainThread : INFO : EPOCH 239 - PROGRESS: at 91.97% examples, 136895 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:48:53,963 : MainThread : INFO : worker thread finished; awaiting finish of 3 more threads
2019-03-11 14:48:53,990 : MainThread : INFO : worker thread finished; awaiting finish of 2 more threads
2019-03-11 14:48:54,004 : MainThread : INFO : worker thread finished; awaiting finish of 1 more threads
2019-03-11 14:48:54,035 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-03-11 14:48:54,037 : MainThread : INFO : EPOCH - 239 : training on 

2019-03-11 14:49:37,520 : MainThread : INFO : EPOCH 244 - PROGRESS: at 26.22% examples, 133626 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:49:38,527 : MainThread : INFO : EPOCH 244 - PROGRESS: at 33.62% examples, 134822 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:49:39,530 : MainThread : INFO : EPOCH 244 - PROGRESS: at 41.23% examples, 136068 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:49:40,531 : MainThread : INFO : EPOCH 244 - PROGRESS: at 53.91% examples, 137220 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:49:41,591 : MainThread : INFO : EPOCH 244 - PROGRESS: at 65.54% examples, 137503 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:49:42,621 : MainThread : INFO : EPOCH 244 - PROGRESS: at 79.70% examples, 137366 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:49:43,633 : MainThread : INFO : EPOCH 244 - PROGRESS: at 90.49% examples, 137666 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:49:44,431 : MainThread : INFO : worker thread finished; awaiting finish of 3 more threads


2019-03-11 14:50:25,965 : MainThread : INFO : EPOCH 249 - PROGRESS: at 7.61% examples, 119800 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:50:27,000 : MainThread : INFO : EPOCH 249 - PROGRESS: at 17.34% examples, 125309 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:50:28,055 : MainThread : INFO : EPOCH 249 - PROGRESS: at 25.16% examples, 126833 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:50:29,096 : MainThread : INFO : EPOCH 249 - PROGRESS: at 32.98% examples, 130377 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:50:30,114 : MainThread : INFO : EPOCH 249 - PROGRESS: at 40.38% examples, 131270 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:50:31,122 : MainThread : INFO : EPOCH 249 - PROGRESS: at 53.28% examples, 134347 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:50:32,138 : MainThread : INFO : EPOCH 249 - PROGRESS: at 63.42% examples, 133637 words/s, in_qsize 7, out_qsize 0
2019-03-11 14:50:33,185 : MainThread : INFO : EPOCH 249 - PROGRESS: at 77.80% examples, 134848 words/s, i

Saving model  models/doc2vec.model


2019-03-11 14:50:46,220 : MainThread : INFO : saved models/doc2vec.model


# Using the doc2vec model

In [14]:
fullname = path.join(utils.Config.models_dir,utils.Config.nameModel)
print("Loading model ", fullname)
modelDoc2Vec = doc2vec.Doc2Vec.load(fullname)
print("The model contains {0} vectors.".format(len(modelDoc2Vec.docvecs)))

2019-03-11 14:52:09,844 : MainThread : INFO : loading Doc2Vec object from models/doc2vec.model


Loading model  models/doc2vec.model


2019-03-11 14:52:10,065 : MainThread : INFO : loading wv recursively from models/doc2vec.model.wv.* with mmap=None
2019-03-11 14:52:10,067 : MainThread : INFO : loading docvecs recursively from models/doc2vec.model.docvecs.* with mmap=None
2019-03-11 14:52:10,068 : MainThread : INFO : loading vocabulary recursively from models/doc2vec.model.vocabulary.* with mmap=None
2019-03-11 14:52:10,069 : MainThread : INFO : loading trainables recursively from models/doc2vec.model.trainables.* with mmap=None
2019-03-11 14:52:10,070 : MainThread : INFO : loaded models/doc2vec.model


The model contains 473 vectors.


In [15]:
"danna" in vocab_dict

False

In [16]:
# just a little test
w1 = "local-search"
words = modelDoc2Vec.wv.most_similar(w1,topn=50)
print("SIM {0:10s} = {1}".format(w1,[w[0] for w in words]))
print("-"*80)

w1 = "local-branching"
words = modelDoc2Vec.wv.most_similar(w1,topn=50)
print("SIM {0:10s} = {1}".format(w1,[w[0] for w in words]))
print("-"*80)

w1 = "relaxation-induced-neighborhood-search"
words = modelDoc2Vec.wv.most_similar(w1,topn=50)
print("SIM {0:10s} = {1}".format(w1,[w[0] for w in words]))
print("-"*80)

print("SIMIL = ", modelDoc2Vec.n_similarity(["variable-neighborhood-search"],["relaxation-induced-neighborhood-search"]))
#print(modelDoc2Vec.docvecs.doctags)
print(modelDoc2Vec.predict_output_word(context_words_list=corpus[5][0], topn=20)) #probability distribution of the center word given context words.
 #     .distance("hybrid", "matheuristic"))

2019-03-11 14:52:16,495 : MainThread : INFO : precomputing L2-norms of word weight vectors


SIM local-search = ['dive', 'placement', 'decoding', 'hsi', 'compilation', 'auction', 'forecasting', 'real', 'scratch', 'very-large-scale-neighborhood-search', 'fixation', 'clock', 'optimization', 'tabu-search', 'rule', 'par', 'bank', 'consecutively', 'happening', 'solution', 'atlanta', 'stakeholder', 'acting', 'higher', 'tel', 'mitra', 'plu', 'fixing', 'image', 'prevention', 'ity', 'available', 'lower-bounding', 'weekend', 'contributed', 'create', 'density', 'physiological', 'observed', 'sort', 'resulted', 'smr', 'including', 'istc', 'variant', 'documentation', 'seemingly', 'closing', 'troya', 'front']
--------------------------------------------------------------------------------
SIM local-branching = ['released', 'bfs', 'kernel', 'kor', 'cri', 'exemplary', 'amenable', 'patch', 'stat', 'din', 'arg', 'cpe', 'recorded', 'instinctive', 'aggregated', 'callback', 'forcing', 'period', 'specified', 'interval', 'logic', 'detour', 'til', 'unweighted', 'jpf', 'prospective', 'caregiver', 'sack

/home/mcaserta/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).


AttributeError: 'Doc2Vec' object has no attribute 'predict_output_word'

In [17]:
result = modelDoc2Vec.most_similar(positive=['bender', 'dantzig-wolfe', 'benders', 'lagrangean', 'relaxation'])#
#result = modelDoc2Vec.most_similar(positive=['local-branching', 'corridor-method', 'kernel-search', 'popmusic'],topn=5)
#result = modelDoc2Vec.most_similar(positive=['lagrangean', 'dantzig-wolfe'], negative=['matheuristic'])
print("result = ", result)

result =  [('tree', 0.22997766733169556), ('regional', 0.22558355331420898), ('counterpart', 0.21651193499565125), ('directional', 0.21186226606369019), ('trois', 0.2084837555885315), ('mould', 0.20721790194511414), ('operate', 0.20072785019874573), ('duplicating', 0.19802889227867126), ('predator', 0.1934083104133606), ('pattern', 0.19275805354118347)]


/home/mcaserta/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [52]:
modelDoc2Vec.wmdistance(corpus[0][0], corpus[1][0])

2019-03-06 17:22:06,826 : MainThread : INFO : Removed 36 and 33 OOV words from document 1 and 2 (respectively).
2019-03-06 17:22:06,827 : MainThread : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-03-06 17:22:06,832 : MainThread : INFO : built Dictionary(1009 unique tokens: ['imply', 'cantly', 'disposed', 'available', 'cited']...) from 2 documents (total 4263 corpus positions)


KeyboardInterrupt: 

In [ ]:
v1=modelDoc2Vec.docvecs[corpus[0][1]]
v2=modelDoc2Vec.docvecs[corpus[1][1]]
print(corpus[1][1])
# use 1-similarity to get a distance
modelDoc2Vec.docvecs.similarity(corpus[0][1][0], corpus[1][1][0]) # note: get the tag, not the list
#modelDoc2Vec.docvecs.similarity(str(0), str(1))

# Populate Lists

We want to create two types of lists:
* List of bi-grams, three-grams, etc. Examples of bi-grams are population-based, corridor-method, etc.
* List of acronyms, e.g., TS, CM ,GA, RINS, etc.

In both cases, we will transform the text into the format "tabu-search", corridor-method". This implies that we need to check any two consecutive words (to indentify cases in which "tabu search" is used) as well as capital-letter acronyms (to identify cases in which "TS" is used.)

In [ ]:
def populate_ngrams(chunk, fNgrams, fAcrons, ngrams_dict, ngrams_new, acrons_dict, acrons_new):
    chunk = re.sub('[“”;\',.’´`\(\{\[\/\]\}\)=+]','', chunk)
    words = [w for w in word_tokenize(chunk) if w not in stop_words]
    for w in words:
        if w.isupper() and w.isalpha() and len(w) > 1 and w not in acrons_dict and w not in acrons_new:
            acrons_new.add(w)
            fAcrons.write(w+"\n")
    #words = [w.lower() for w in word_tokenize(chunk) if w not in stop_words]
    words = [w.lower() for w in words]
    i = 0
    corrected = []
    while i < len(words)-1: # skip last element (we could lose a few words here)
        w = words[i]
        if len(w) <= 1:
            i += 1
            continue
        if w[-1] == "-" and w[:-1].isalpha() and words[i+1].isalpha() and len(w[:-1]) >= 1:
            w = w[:-1] + words[i+1]
            i += 1
            print("** Concatenated word = ", w)
        # if we find a word that contains a hyphen, it must be a composite word, e.g. "worst-case"
        if len(re.findall("-",w)) > 0 and w[0] != "-" and w[-1] != "-" and w not in ngrams_dict and w not in ngrams_new:
            print("Adding '{0}' to list of n-grams".format(w))
            fNgrams.write(w+"\n") # add it to our list of n-grams
            ngrams_new.append(w)
            
        if (len(re.findall("-",w)) > 0 or w.isalpha()) and len(w) > 2: # eliminate short words
            corrected.append(w)
        i += 1
        
    return corrected
       


In [ ]:
ngrams_dict = create_dict(ngramsFile)
acrons_dict = create_dict(acronsFile)
ngrams_new  = []
acrons_new = {""} #"CM", "TS", "GA", "ACO", "POPMUSIC", "VNS", "VNDS", "RINS"} # a set


fNgrams = open(ngramsFile, "a") # use this in the first run, to populate the file
fAcrons = open(acronsFile, "a") # list of acronyms

#pdfFile = 'biblio/summary.pdf'
pdfFile = 'biblio/puchinger-05.pdf'

document = open(pdfFile, 'rb')
#Create resource manager
rsrcmgr = PDFResourceManager()
# Set parameters for analysis.
laparams = LAParams()
# Create a PDF page aggregator object.
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)

doc = []
biblio = False
for page in PDFPage.get_pages(document):
    interpreter.process_page(page)
    # receive the LTPage object for the page.
    layout = device.get_result()
    for element in layout:
        if isinstance(element, LTTextBoxHorizontal):
            el = element.get_text()
            words = populate_ngrams(el, fNgrams, fAcrons, ngrams_dict, ngrams_new, acrons_dict, acrons_new)
            
            # next, attempt to skip bibliography
            if len(words) == 0 and "Bibliography" in el:
                print("EL = ", el)
                print("Found Biblio? STOP Here" )
                biblio = True
                break
                
            for w in words:
                doc.append(w)
    if biblio:
        break
    #if i > 1:
    #    break
        
fNgrams.close()
fAcrons.close()

In [ ]:
acrons

In [ ]:
A = {"CM"}
A.add(1)
A